In [1]:
from transformers import AutoModel, AutoConfig

In [7]:
model_name = "microsoft/deberta-v3-large"
model_name = "microsoft/deberta-v2-xxlarge"

model_cfg = AutoConfig.from_pretrained(model_name)
model_cfg.is_encoder_decoder, model_cfg.is_decoder

(False, False)

In [8]:
model_cfg.architectures

In [9]:
model = AutoModel.from_pretrained(model_name)
print(f"# params: {sum(p.numel() for p in model.parameters()):,d}")

Some weights of the model checkpoint at microsoft/deberta-v2-xxlarge were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# params: 1,564,549,632
